# RAG EXAMPLE

In [1]:
from typing import Dict, List

import numpy
import torch

from inference import (
    optimize_memory,
    setup_tokenizer_and_model,
    get_model_output,
)
from utils.chunkify_docx import DocxChunker
from utils.db_types import VectorDBType
from utils.embedder import embedd_sequences

from vector_db.milvus_connection import SimpleVectorDB
from vector_db.chroma_connection import insert_documents_chroma, get_documents_chroma, get_collection

QueryResult = Dict[str, List[str] | List[float]]

/home/snojj/Desktop/FAKS/tup/rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def chunkify_document(
    docx_path: str, chunk_size: int = 200, chunk_overlap: int = 40
) -> List[str]:
    chunker = DocxChunker(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunker.process_docx(docx_path)

    return chunks


def _embed_chunks(chunks: List[str]) -> numpy.ndarray:
    embeddings = embedd_sequences(chunks)

    return embeddings.numpy()

In [3]:
def _embed_chunks(chunks: List[str]) -> numpy.ndarray:
    embeddings = embedd_sequences(chunks)

    return embeddings


def store_embedding_vectors_in_vector_db(
    chunks: List[str], db_type: VectorDBType
) -> None:

    if db_type == VectorDBType.MILVUS:
        # Store vectors in Milvus

        # Initialize the vector database
        db = SimpleVectorDB()

        embedding_vectors = _embed_chunks(chunks)

        # Insert vectors and metadata
        db.insert(embedding_vectors, chunks)
        print(f"Inserted {len(chunks)} vectors")

        db.close()

    elif db_type == VectorDBType.CHROMA:
        # Store vectors in Chroma

        collection = get_collection("test_rag")

        insert_documents_chroma(collection, documents=chunks)

    else:
        raise ValueError(f"Unsupported vector DB type: {db_type}")


In [4]:
def _handle_chroma_query_results(results) -> QueryResult:
    ids, distances, documents = (
        results["ids"],
        results["distances"],
        results["documents"],
    )

    return {"ids": ids, "distances": distances, "documents": documents}


def query_vector_db(query: str, db_type: VectorDBType, top_k: int = 3) -> QueryResult:

    if db_type == VectorDBType.MILVUS:
        # Store vectors in Milvus

        # Initialize the vector database
        db = SimpleVectorDB()

        query_embedding = _embed_chunks([query])[0]

        # Perform a search
        results = db.search(query_embedding, top_k=top_k)

        # Close connection
        db.close()

        return results

    elif db_type == VectorDBType.CHROMA:
        # Store vectors in Chroma
        
        collection = get_collection("test_rag")

        results = get_documents_chroma(
            collection, query_texts=[query], top_k=top_k
        )

        return _handle_chroma_query_results(results)
    else:
        raise ValueError(f"Unsupported vector DB type: {db_type}")


In [5]:
def clear_collection(db_type: VectorDBType) -> QueryResult:

    if db_type == VectorDBType.MILVUS:
        # Store vectors in Milvus

        # Initialize the vector database
        db = SimpleVectorDB()

        db.delete()

        # Close connection
        db.close()


    elif db_type == VectorDBType.CHROMA:
        # Store vectors in Chroma
        
        collection = get_collection("test_rag")
        
        
        # delete_collection_chroma("test_rag")

    else:
        raise ValueError(f"Unsupported vector DB type: {db_type}")


In [6]:
def clean_response(response: str, original_prompt: str) -> str:
    # if "[end]" in response:
    #     return response.split("[end]", 1)[1].strip()

    if response.startswith(original_prompt):
        return response[len(original_prompt) :].strip().lstrip(".")

    return response

In [7]:
def run_inference(question: str, context: str = None) -> str:

    prompt: str
    if context:
        prompt = f"""This is a segment from the Slovenian costitution: "{context}". Using this information can you answer the following question in one sentence: "{question}"."""
    else:
        prompt = f"""Anwser the following question in one sentence: "{question}"."""

    optimize_memory()

    tokenizer, model = setup_tokenizer_and_model()

    try:
        response = get_model_output(prompt=prompt, model=model, tokenizer=tokenizer)

        response = clean_response(response, original_prompt=prompt)

        return response
    except RuntimeError as e:
        if "out of memory" in str(e):
            torch.cuda.empty_cache()
            print("GPU out of memory, try clearing the cache...")
            raise e
        else:
            raise e

In [ ]:
user_query = (
    "What are the rights to asylum in Slovenia?"
)
response = run_inference(user_query)
print("Response without rag: ", response)

# Example usage
docx_path = "../media/Constitution.docx"
chunks = chunkify_document(docx_path)


store_embedding_vectors_in_vector_db(chunks, db_type=VectorDBType.CHROMA)

results = query_vector_db(user_query, db_type=VectorDBType.CHROMA, top_k=2)

response = run_inference(user_query, context=results["documents"][0])
print("Response with rag: ", response)

clear_collection(db_type=VectorDBType.CHROMA)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ebe35119010>>
Traceback (most recent call last):
  File "/home/snojj/Desktop/FAKS/tup/rag/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.50s/it]
